In [ ]:
import os
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from pathlib import Path
from datasets import load_dataset
from tqdm import tqdm

%pip install transformers


In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from datasets import load_dataset
from transformers import GPT2Tokenizer
from tqdm import tqdm


import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleGPT(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, hidden_dim, max_length):
        super(SimpleGPT, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_length, embed_size))
        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads, dim_feedforward=hidden_dim)
            for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(embed_size, vocab_size)
    
    def forward(self, x, mask=None):
        # Embedding + Positional Encoding
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        
        # Ensure mask is the correct shape (batch_size, seq_len)
        if mask is not None:
            mask = mask.bool()  # Convert mask to boolean if it's not already
        
        # Transpose input for transformer (seq_len, batch_size, embed_size)
        x = x.transpose(0, 1)
        
        # Pass through transformer layers
        for transformer in self.transformer_blocks:
            x = transformer(x, src_key_padding_mask=mask)
        
        # Transpose back to (batch_size, seq_len, embed_size)
        x = x.transpose(0, 1)
        
        # Output layer
        logits = self.fc_out(x)
        return logits



# Ensure directories exist
os.makedirs("./reviewcar", exist_ok=True)
os.makedirs("./tokenizer_en", exist_ok=True)
os.makedirs("./tokenizer_my", exist_ok=True)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset
dataset = load_dataset("florentgbelidji/car-reviews")

# Preprocessing: Convert to DataFrame
import pandas as pd
df = pd.DataFrame(dataset['train'])

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

# Convert 'Rating' to numeric
df['Rating'] = pd.to_numeric(df['Rating'])

# Define custom Dataset
class CarReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        review = str(self.data['Review'][index])
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        
        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Fix padding issue by assigning pad_token
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set

# Create dataset
max_length = 128
dataset = CarReviewDataset(df, tokenizer, max_length)

# Split dataset into train, validation, and test sets
train_size = 0.7
val_size = 0.15
test_size = 0.15

train_length = int(train_size * len(dataset))
val_length = int(val_size * len(dataset))
test_length = len(dataset) - train_length - val_length

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_length, val_length, test_length])

# DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model Definition
# Model Initialization
vocab_size = tokenizer.vocab_size
embed_size = 256
num_heads = 8
num_layers = 4
hidden_dim = 512

model = SimpleGPT(vocab_size, embed_size, num_heads, num_layers, hidden_dim, max_length).to(device)




# Loss and Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, mask=attention_mask)
        loss = criterion(outputs.view(-1, vocab_size), input_ids.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}')

    # Validation Loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, mask=attention_mask)
            loss = criterion(outputs.view(-1, vocab_size), input_ids.view(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Val Loss: {avg_val_loss:.4f}')
    
    # Step the learning rate scheduler
    scheduler.step()


In [ ]:
# Save the model
model_path = "car_gpt_model.pth"
torch.save(model.state_dict(), model_path)

# Save the tokenizer
tokenizer.save_pretrained('./tokenizer')
# Load the model
model = SimpleGPT(vocab_size, embed_size, num_heads, num_layers, hidden_dim, max_length)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('./tokenizer')

# Ensure pad token is set correctly
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
def generate_text(model, tokenizer, prompt, max_length=50, temperature=1.0, top_k=50):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    generated = input_ids

    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(generated)
            next_token_logits = outputs[:, -1, :] / temperature
            
            # Apply top-k sampling
            top_k_values, top_k_indices = torch.topk(next_token_logits, top_k, dim=-1)
            next_token_probs = torch.softmax(top_k_values, dim=-1)
            
            # Sample from the top-k options
            next_token_id = torch.multinomial(next_token_probs, num_samples=1).squeeze(1)
            
            # Append the new token to the generated sequence
            generated = torch.cat((generated, top_k_indices.gather(1, next_token_id.unsqueeze(1))), dim=1)

            # Stop if end-of-sequence token is generated
            if next_token_id.item() == tokenizer.eos_token_id:
                break
    
    output_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    return output_text

# Example usage
prompt = "I want a reliable car with good fuel efficiency."
generated_text = generate_text(model, tokenizer, prompt, max_length=100, temperature=0.7, top_k=50)
print(generated_text)
